# IMPORT

In [1]:
import os, shutil #Controlling Files and paths and folder control
from shutil import copyfile
from pdf2image import convert_from_path #Libary convert pdf file to img file
from google.oauth2 import service_account #Control API Keys
from google.cloud import vision # Vision API from Google
import io
import string, random
import pandas as pd
# Imports the Google Cloud client library
from google.cloud import language_v1
from google.cloud.language_v1 import enums

In [2]:
#from __future__ import unicode_literals, print_function
#from spacy.lang.en import English # updated

# Paths

In [3]:
#Paths that have all folders which use everything
pdfIMGPopplerPath = '/Users/kunal/Documents/VdartResumeProject/Poppler/poppler-0.68.0_x86/poppler-0.68.0/bin/'
imgTxtVisionAPIPath = "/Users/kunal/Documents/VdartResumeProject/APIKEYSGOOGLE/resumeMatcher-pdf2img.json"
runningDocumentPath = '/Users/kunal/Documents/VdartResumeProject/runningDoc/'
sourceFolderResumesPath = '/Users/kunal/Documents/VdartResumeProject/50_resumes/'
excelFilesPath = '/Users/kunal/Documents/VdartResumeProject/ExcelFiles/'
nlpAutoAPIPath = "/Users/kunal/Documents/VdartResumeProject/APIKEYSGOOGLE/resumeMatcher-NLP_create_data.json"
jsonFolderPath = '/Users/kunal/Documents/VdartResumeProject/JSONLFILES/'

## Extract Excel and Turn into a list

In [4]:
os.chdir(excelFilesPath)

In [5]:
text = pd.ExcelFile(EXCELFILENAMESent)
totalTextRunning = runningTextFINAL
attributes = text.parse("Sheet1") 
# read excel

NameError: name 'EXCELFILENAMESent' is not defined

In [ ]:
attributes
# this is a table of all of the entities. 
# IMPORTANT -- the name of the first row with the text must be named "Text"
# could be every how long columns

In [46]:
RowList = attributes.values.tolist()
attributeList = [[i for i in row if isinstance(i,str)] for row in RowList]
#Converts to a 2d array 

## Create Final List

In [47]:
def sentence_2_word(sentence):
    # this functuion takes a sentence and splits it into each word and returns a list of all the words
    word = ""
    listofWords = []
    for i in sentence:
        if not (i == " " or i == "\n"):
            word += i 
        else:
            listofWords.append(word)
            word = ""
    if len(listofWords) == 0:
        listofWords.append(sentence)
    return listofWords

In [48]:
def find_all_indexes(input_str, search_str):
    # find all the positions of where a text is in a string
    # not used so dont worry
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1[0]

In [49]:
def checkNotWorkingValues(textFindPosition, totalTextRunning):
    # if the find function cant find the text in the total text then this function splits the search text into each word 
    # find its value out of the entire text and if they are close enough then then it returns the position of the starting 
    # word of the search text but if it can't get a definitie answer then it returns -1 
    words = textFindPosition.split()
    wordPositions = []
    for singleWord in words:
        wordPositions.append(totalTextRunning.find(singleWord))
    positionsCount = 0
    numCorrect = 0
    for num in range(len(wordPositions)-1):
        if wordPositions[num]+len(words[num])+1 == wordPositions[num+1]:
            numCorrect+=1
        positionsCount+=1
    if numCorrect == len(words)-1:
        return wordPositions[0]
    else:
        return -1

In [ ]:
def checkNotWorkingValues(textFindPosition, totalTextRunning):
    words = textFindPosition.split()
    if len(words) >= 7:
        wordListNew = []
        for number in range(1, len(words), 2):
            wordListNew.append(words[number-1] + " " + words[number])
        if not len(words) % 2 == 0: 
            wordListNew.append(words[-1])
        #print(wordListNew)
    words = wordListNew
    wordPositions = []
    numberstarting = 0  
    for singleWord in words:
        #print(totalTextRunning[numberstarting:].find(singleWord))
        wordPositions.append(totalTextRunning[numberstarting:].find(singleWord)+numberstarting)
        if not totalTextRunning[numberstarting:].find(singleWord) == -1:
            numberstarting = totalTextRunning[numberstarting:].find(singleWord) +numberstarting
        #print(numberstarting)    
    #print(wordPositions)
    positionsCount = 0
    numCorrect = 0
    for num in range(len(wordPositions)-1):
        if abs(wordPositions[num]+len(words[num])+1 - wordPositions[num+1]) < 2:
            numCorrect+=1
        #else:
            #print(words[num])
            #print(wordPositions[num])
            #print(len(words[num])+1)
            #print(wordPositions[num]+len(words[num])+1)
            #print(wordPositions[num+1])
            #print("")
        positionsCount+=1
    #print(positionsCount)
    #print(numCorrect)
    if numCorrect/positionsCount > 0.6:
        return wordPositions[0]
    else:
        return -1

In [50]:
numbAttributeReal = 0
for line in attributeList:
    #value_when_true if condition else value_when_false
    numbAttributeReal += int((len(line)-1)/2) if (len(line)-1)/2 != 0.5 else 0
# calculates the number of attributes in the table/matrix -- simple maths

In [51]:
print("Number of attributes detected is: " + str(numbAttributeReal))
# simple print

Number of attributes detected is: 36


In [52]:
# goes through each row(sentence) and extract position of each attribute
attributesNumberList = []
for j in attributeList:
    runningAttributeNum = []
    if len(j) == 2:
        continue # would be like ----- "[sentence] - none" --- no attribute in it
    if j[2] == "All": # if the value is at all then there is only 1 attribute and its the entire text 
        # searchs for the entire text for the entire word
        startChar = totalTextRunning.find(j[0].strip())
        if startChar == -1:
            #checks for the not working -- check function
            startChar = checkNotWorkingValues(j[0].strip(), totalTextRunning)
        if startChar == -1:
            # lower everything and see if it can find it then
            startChar = totalTextRunning.lower().find(j[0].lower().strip())
        endChar = startChar + len(j[0].strip())
        runningAttributeNum.extend([j[1].strip().upper() , startChar, endChar])
        attributesNumberList.append(runningAttributeNum)
    else:
        #print("RUNNING " + str(int((len(j)-1)/2)))
        # im not going to go in detail explaining this but copy this function into a sepreate file and try it out
        # similar to the "all thing" but it works for any number of attributes 
        for w in range(0, int((len(j)-1)), 2):
            runningAttributeNum = []
            if not totalTextRunning.find(j[2+w]) == -1:
                startChar = totalTextRunning.find(j[2+w].strip())
                if startChar == -1:
                    startChar = checkNotWorkingValues(j[2+w].strip(), totalTextRunning)
                if startChar == -1:
                    startChar = totalTextRunning.lower().find(j[2+w].lower().strip())
                endChar = startChar + len(j[2+w])
                runningAttributeNum.extend([j[1+w].upper(), startChar, endChar])
            else:
                words = sentence_2_word(j[2+w]) # for the double check
                detected = 0
                for g in words:
                    if not j[2+w].find(g) == -1:
                        #charTemp = j[2+w].find(g)
                        detected+=1
                if detected/len(words) > 0.95: # split words and 0.95 is threshold
                    startChar = totalTextRunning.find(words[0])
                    endChar = startChar + len(words[0])
                    runningAttributeNum.extend([words[0].upper(), startChar, endChar])
                else:
                    charTemp = -1 # couldn't finf anything
                    print("NOT DETECTED")
                    endChar , startChar = -1
                    runningAttributeNum.extend(["NOTDETECTED", startChar, endChar])
            attributesNumberList.append(runningAttributeNum)

In [53]:
len(attributesNumberList)

36

In [54]:
attributesNumberList

[['NAME', 0, 12],
 ['ADDRESS', 13, 29],
 ['EMAIL', 30, 53],
 ['PHONE', 54, 66],
 ['WORKEXPERIENCE', 83, 117],
 ['COMPANY', 118, 141],
 ['DATE', 142, 169],
 ['WORKEXPERIENCE', -1, 117],
 ['WORKEXPERIENCE', 289, 352],
 ['WORKEXPERIENCE', -1, 146],
 ['WORKEXPERIENCE', 501, 560],
 ['WORKEXPERIENCE', 561, 588],
 ['COMPANY', 589, 627],
 ['DATE', 628, 653],
 ['WORKEXPERIENCE', -1, 171],
 ['WORKEXPERIENCE', -1, 170],
 ['WORKEXPERIENCE', 999, 1014],
 ['ADDRESS', 1015, 1027],
 ['DATE', 1028, 1052],
 ['WORKEXPERIENCE', -1, 103],
 ['WORKEXPERIENCE', -1, 145],
 ['WORKEXPERIENCE', 1305, 1383],
 ['WORKEXPERIENCE', 1384, 1442],
 ['EDUCATION', 1453, 1472],
 ['EDUCATION', 589, 627],
 ['DATE', 645, 653],
 ['EDUCATION', 1521, 1528],
 ['EDUCATION', 1529, 1567],
 ['DATE', 1028, 1032],
 ['SKILL', 1580, 1590],
 ['SKILL', 1591, 1613],
 ['SKILL', 1614, 1626],
 ['SKILL', 1580, 1590],
 ['SKILL', 1638, 1652],
 ['SKILL', 1653, 1664],
 ['SKILL', 1614, 1626]]

In [ ]:
len(attributesNumberList)

In [55]:
for testing in attributesNumberList:
    # if it can't find it removes it 
    if testing[1] == -1:
        print(testing)
        attributesNumberList.remove(testing)

['WORKEXPERIENCE', -1, 117]
['WORKEXPERIENCE', -1, 146]
['WORKEXPERIENCE', -1, 171]
['WORKEXPERIENCE', -1, 103]


In [125]:
textFindPosition = textg
words = textFindPosition.split()
wordPositions = []
numberstarting = 0  
for singleWord in words:
    #print(totalTextRunning[numberstarting:].find(singleWord))
    wordPositions.append(totalTextRunning[numberstarting:].find(singleWord)+numberstarting)
    if not totalTextRunning[numberstarting:].find(singleWord) == -1:
        numberstarting = totalTextRunning[numberstarting:].find(singleWord) +numberstarting
    #print(numberstarting)    
#print(wordPositions)
positionsCount = 0
numCorrect = 0
for num in range(len(wordPositions)-1):
    if abs(wordPositions[num]+len(words[num])+1 - wordPositions[num+1]) < 2:
        numCorrect+=1
    #else:
        #print(words[num])
        #print(wordPositions[num])
        #print(len(words[num])+1)
        #print(wordPositions[num]+len(words[num])+1)
        #print(wordPositions[num+1])
        #print("")
    positionsCount+=1
#print(positionsCount)
#print(numCorrect)
if numCorrect/positionsCount > 0.6:
    print(True)

[170, 180, 182, 190, 193, 194, 194, 204, 213, 214, 225, 234, 243, 249, 256, 264, 269, 271, 277, 280, 279]
True


In [126]:
textFindPosition = textg
words = textFindPosition.split()
print(words)
if len(words) >= 7:
    wordListNew = []
    for number in range(1, len(words), 2):
        wordListNew.append(words[number-1] + " " + words[number])
    if not len(words) % 2 == 0: 
        wordListNew.append(words[-1])
    
    print(wordListNew)

['Performed', 'a', 'variety', 'of', 'tasks', 'as', 'a', 'material', 'handler', 'and', 'forklift', 'operator', 'while', 'always', 'working', 'with', 'a', 'sense', 'of', 'urgency', 'word.']
['Performed a', 'variety of', 'tasks as', 'a material', 'handler and', 'forklift operator', 'while always', 'working with', 'a sense', 'of urgency', 'word.']


In [127]:
words = wordListNew

In [128]:
textFindPosition = textg
words = textFindPosition.split()
print(words)
if len(words) >= 7:
    wordListNew = []
    for number in range(1, len(words), 2):
        wordListNew.append(words[number-1] + " " + words[number])
    if not len(words) % 2 == 0: 
        wordListNew.append(words[-1])
    #print(wordListNew)
words = wordListNew
wordPositions = []
numberstarting = 0  
for singleWord in words:
    #print(totalTextRunning[numberstarting:].find(singleWord))
    wordPositions.append(totalTextRunning[numberstarting:].find(singleWord)+numberstarting)
    if not totalTextRunning[numberstarting:].find(singleWord) == -1:
        numberstarting = totalTextRunning[numberstarting:].find(singleWord) +numberstarting
    #print(numberstarting)    
#print(wordPositions)
positionsCount = 0
numCorrect = 0
for num in range(len(wordPositions)-1):
    if abs(wordPositions[num]+len(words[num])+1 - wordPositions[num+1]) < 2:
        numCorrect+=1
    #else:
        #print(words[num])
        #print(wordPositions[num])
        #print(len(words[num])+1)
        #print(wordPositions[num]+len(words[num])+1)
        #print(wordPositions[num+1])
        #print("")
    positionsCount+=1
#print(positionsCount)
#print(numCorrect)
if numCorrect/positionsCount > 0.6:
    print(True)

True


In [100]:
totalTextRunning[170:279]

'Performed a variety of tasks as a material handler and forklift operator while always working with a\nsense of'

In [96]:
print(t)

170
180


In [112]:
wordPositions = []
numberstarting = 0  
for singleWord in words:
    #print(totalTextRunning[numberstarting:].find(singleWord))
    wordPositions.append(totalTextRunning[numberstarting:].find(singleWord)+numberstarting)
    if not totalTextRunning[numberstarting:].find(singleWord) == -1:
        numberstarting = totalTextRunning[numberstarting:].find(singleWord) +numberstarting
    #print(numberstarting)    
print(wordPositions)
positionsCount = 0
numCorrect = 0
print("")
for num in range(len(wordPositions)-1):
    print(wordPositions[num])
    print(len(words[num])+1)
    print(wordPositions[num]+len(words[num])+1)
    
    print(wordPositions[num+1])
    if abs(wordPositions[num]+len(words[num])+1 - wordPositions[num+1]) < 2:
        numCorrect+=1
    positionsCount+=1
    print("")
print(numCorrect)
print(len(words))
if numCorrect == len(words)-1:
    print(wordPositions[0])


[170, 180, 182, 190, 193, 194, 194, 204, 213, 214, 225, 234, 243, 249, 256, 264, 269, 271, 277, 280, 279]

170
10
180
180

180
2
182
182

182
8
190
190

190
3
193
193

193
6
199
194

194
3
197
194

194
2
196
204

204
9
213
213

213
8
221
214

214
4
218
225

225
9
234
234

234
9
243
243

243
6
249
249

249
7
256
256

256
8
264
264

264
5
269
269

269
2
271
271

271
6
277
277

277
3
280
280

280
8
288
279

14
21


In [62]:
totalTextRunning.find(textg[0:10].strip())

170

In [61]:
startChar = totalTextRunning.find(textg.strip())
if startChar == -1:
    #checks for the not working -- check function
    startChar = checkNotWorkingValues(textg.strip(), totalTextRunning)
    print(startChar)
if startChar == -1:
    # lower everything and see if it can find it then
    startChar = totalTextRunning.lower().find(textg.lower().strip())
    print(startChar)
endChar = startChar + len(textg.strip())
print([textg.strip().upper() , startChar, endChar])

-1
-1
['PERFORMED A VARIETY OF TASKS AS A MATERIAL HANDLER AND FORKLIFT OPERATOR WHILE ALWAYS WORKING WITH A SENSE OF URGENCY.', -1, 117]


## DataSet to JSON FILE

In [ ]:
len(attributesNumberList)

In [ ]:
attributeCount = len(attributesNumberList)
text = totalTextRunning
attributes = attributesNumberList

In [ ]:
text

In [ ]:
def createJSONFILE(attributeCount, text, attributes):
    #the 2 string named "totalString" and "attributeString" are just formated in a way how google likes their input value
    # loops through everything and uses the replace function to replace it with all the attributes
    # you have to test this out yourself to understand how it works
    alphab = list(string.ascii_lowercase)
    for alpa in range(int(attributeCount/26)):
        alphab.extend(list(string.ascii_lowercase))
    totalString = """{"textSnippet":{"content":"textCODE"},"annotations":[attributeCODE]}"""
    attributeString = """{"displayName":"AtribnameCODE","textExtraction":{"textSegment":{"startOffset":"sCharCODE","endOffset":"eCharCODE"}}}"""
    totalString =  totalString.replace("textCODE", text)
    tempStringReplacer = ""
    for i in range(attributeCount):
        if i != attributeCount-1:
            tempStringReplacer+=("atriCODE"+str(i+1)+alphab[i]+",")
        else:
            tempStringReplacer+=("atriCODE"+str(i+1)+alphab[i])
    totalString = totalString.replace("attributeCODE", tempStringReplacer)
    for i in range(attributeCount):
        attributeStringNEW = attributeString
        #print(attributes[i])
        attributeStringNEW = attributeStringNEW.replace("AtribnameCODE", attributes[i][0])
        attributeStringNEW = attributeStringNEW.replace("sCharCODE", str(attributes[i][1])).replace("eCharCODE", str(attributes[i][2]))
        totalString = totalString.replace(("atriCODE"+str(i+1)+alphab[i]), attributeStringNEW)
    return totalString

## Final Run Create JSON

In [ ]:
file = createJSONFILE(attributeCount, text, attributes)

In [ ]:
attributeCount

In [ ]:
file

## Save JSONL FILE

In [ ]:
os.chdir(jsonFolderPath)
# open a file and save it 
jsonFileName = "FINAL_"+documentRun[9:-4]+".jsonl"
text_file = open(jsonFileName, "w")
text_file.write(file)
text_file.close()

In [ ]:
print(jsonFileName)

In [ ]:
f = open(jsonFileName, "r")
#print(f.read())

Now your done and you have the Json file, give it to kunal and he will put it into GCP platform account